# NYC Cab Trip Duration Prediction

### Importing Necessary Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
from warnings import filterwarnings
filterwarnings("ignore")


In [2]:
df1=pd.read_parquet('yellow_tripdata_2022-01.parquet')
df2=pd.read_parquet('yellow_tripdata_2022-02.parquet')
df3=pd.read_parquet('yellow_tripdata_2022-03.parquet')
df4=pd.read_parquet('yellow_tripdata_2022-04.parquet')
df5=pd.read_parquet('yellow_tripdata_2022-05.parquet')
df6=pd.read_parquet('yellow_tripdata_2022-06.parquet')

data=pd.concat([df1.sample(121553) ,df2.sample(121553) ,df3.sample(121553) ,df4.sample(121553) ,df5.sample(121553) ,df6.sample(121553)  ])

FileNotFoundError: [Errno 2] No such file or directory: 'yellow_tripdata_2022-01.parquet'

Reading the data from praquet files for the months of January to June

In [ ]:
data

### Data preprocessing/Munging

In [ ]:

data = data.reset_index()

In [ ]:
data

In [ ]:
data=data.drop(labels=['index'],axis=1)

In [ ]:
data

Read another dataset that contains NYC Taxi Zones

In [ ]:
df=pd.read_csv("taxi_zones.csv")

In [ ]:
df

In [ ]:
from geopy.geocoders import ArcGIS
import folium as f

In [ ]:

loc=ArcGIS()

In [ ]:
nyc=loc.geocode("New York") 

Using geopy.geocoders we fetch the geocodes of New York

In [ ]:
print(nyc.latitude)
print(nyc.longitude)

Above is the latitude and longitude of New York

In [ ]:
# data =df['zone'].sample(50)
#data.apply(loc.geocode)

In [ ]:
df["Location"]=df["zone"].apply(loc.geocode)

Apply geocode to our datasets 'data' and 'zone'

In [ ]:
df["longitude"]=df["Location"].apply(lambda x: x.longitude)
df["latitude"]=df["Location"].apply(lambda x: x.latitude)

Seperating longitude and latitude from location column

In [ ]:
df

In [ ]:
df=df.drop(columns=["OBJECTID","Shape_Leng","the_geom","Shape_Area","borough","Location"])
df

Drop not so important columns

In [ ]:
data

In [ ]:
data1=data.merge(df, left_on='PULocationID',right_on='LocationID')
data1

Merging two datasets to make as one whole dataset

In [ ]:
data1 = data1.rename(columns={'longitude': 'pickup_longitude', 'latitude': 'pickup_latitude'})


In [ ]:
data1=data1.drop(['LocationID','zone'],axis=1)
data1

In [ ]:
data1=data1.merge(df, left_on='DOLocationID',right_on='LocationID')


In [ ]:
data1 = data1.rename(columns={'longitude': 'dropoff_longitude', 'latitude': 'dropoff_latitude'})
data1=data1.drop('LocationID',axis=1)
data1

In [ ]:
data1.to_csv("final_yellow_tripdata.csv")

converted parquet data to csv

In [ ]:
dk_time = pd.to_datetime(data1["tpep_dropoff_datetime"]);
pk_time = pd.to_datetime(data1["tpep_pickup_datetime"]);
D=round(abs(dk_time - pk_time)/np.timedelta64(1,"s") / 60)

data1["trip_duration"]=D
data1

Converting 'tpep_dropoff_datetime' and 'tpep_pickup_datetime' columns from string to date-time format.
Also calculate 'trip_duration' based on pickup and dropoff time

In [ ]:
df=data1.sample(300000)

Restricting data to 30000 rows for faster calculations

In [ ]:
df.sample(20)    # print top view of data set 

In [ ]:
df.tail(20) #bottom view

Drop statasticaly unimportants columns

In [ ]:
df=df.drop(labels=['VendorID','RatecodeID','store_and_fwd_flag'],axis=1)

In [ ]:
df.columns  #columns overview

In [ ]:
df.shape

In [ ]:
df.isna().sum()     

In [ ]:
df=df.dropna()

In [ ]:
df.describe()    #statastical summary 

### Exploratory Data Analysis

In [ ]:
plt.figure(figsize=(10,10))                   #Analysis using heatmap
sb.heatmap(df.corr(),annot=True,fmt='.1f')
plt.show()

Plot heat map to understand how the variables behave with each other. 

It is found that 'trip_duration' and 'fare_amount' are highly correlated. We remove 'fare_amount' to avoid multicollinearity.

In [ ]:
df=df.drop(labels=["fare_amount"],axis=1)    #Removed the column due to high correlation


**Multicollinearity** occurs when two or more independent variables(also known as predictors) are highly correlated with one another in a regression model. 

This means that an independent variable can be predicted from another independent variable in a regression model. In our use case, there is no multicollinearity


In [ ]:
plt.figure(figsize=(20,20))
sb.heatmap(df.corr(),vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5},annot=True)
plt.tight_layout()
plt.show()

Heat map after removing multicollinearity 

In [ ]:
sb.distplot(df['trip_duration'], kde = True, bins = 5)   #Analysis of target colunmn

In [ ]:
sb.boxplot(x="passenger_count", y="trip_duration",data=df);
plt.title("Average Trip Duration for Number of Passengers");
plt.xlabel("passenger_count");
plt.ylabel("Trip Duration");

The plot for 'Average Trip Duration for Number of Passengers' shows more number of cab trips take place with less number of people on board generally with 1 or 2 passengers

In [ ]:
sb.barplot(x="passenger_count", y="trip_duration",data=df);
plt.title("Average Trip Duration for Number of Passengers");
plt.xlabel("passenger_count");
plt.ylabel("Trip Duration");

In [ ]:
df['tpep_pickup_datetime']=pd.to_datetime(df["tpep_pickup_datetime"])
df['tpep_dropoff_datetime']=pd.to_datetime(df["tpep_dropoff_datetime"])

In [ ]:
print(type(df['tpep_pickup_datetime']))
print(type(df['tpep_dropoff_datetime']))

In [ ]:
#Function to convert date-time features into date & time 
def convert_to_date_dtype(Dataframe,col):
   
    Dataframe[col] = pd.to_datetime(Dataframe[col], format= '%d-%m-%Y %H:%M')
    Dataframe[col+'_day'] = Dataframe[col].dt.dayofweek
    Dataframe[col+'_month'] = Dataframe[col].dt.month
    Dataframe[col+'_hour'] = Dataframe[col].dt.hour
    Dataframe[col+'_minute'] = Dataframe[col].dt.minute

Function to convert 'tpep_pickup_datetime' and 'tpep_dropoff_datetime' from object column to datetime column. We can confirm it in the below cell.

In [ ]:

convert_to_date_dtype(df, 'tpep_pickup_datetime')
convert_to_date_dtype(df, 'tpep_dropoff_datetime')

df[['tpep_pickup_datetime', 'tpep_dropoff_datetime']].info()

'tpep_pickup_datetime' and 'tpep_dropoff_datetime' columns are further divided into respective tpep_pickup_datetime_day, tpep_pickup_datetime_month, tpep_pickup_datetime_hour, tpep_pickup_datetime_minute columns and tpep_dropoff_datetime_day, tpep_dropoff_datetime_month, tpep_dropoff_datetime_hour, tpep_dropoff_datetime_minute columns for accurate prediction.

In [ ]:

df

Map for one pickup and dropoff location based on the latitude and longitude of New york

In [ ]:
map=f.Map(location=[40.71455000000003,-74.00713999999994])
map.add_child(f.Marker(location=[40.71455000000003,-74.00713999999994])) #map view of New York city

In [ ]:
import folium

location = [40.730610, -73.935242]
location2=[40.708469,-74.017120]

map=folium.Map(location=location ,width=800,hight=400,zoom_start = 10,)


* Map view for Pickup Points

In [ ]:
pickup =df[["pickup_longitude",'pickup_latitude']]
pickup
data = list(zip(pickup.pickup_latitude.values,
                            pickup.pickup_longitude.values,
                            ))
type(data[0][0])

In [ ]:
pickup = pd.DataFrame(data)
pickup.head()

In [ ]:
from folium.plugins import HeatMap    # import library heatmap
pickup_map = folium.Map(location = location, zoom_start = 10,)

hm_wide = HeatMap( pickup.values,
                     min_opacity= 0.2,
                     radius= 5, blur= 8,
                     max_zoom= 1 
                 )

pickup_map.add_child(hm_wide)

pickup_map

* Map view for Dropoff Points

In [ ]:
dropoff = pd.DataFrame(data)
dropoff=dropoff.dropna()
dropoff.head()

In [ ]:
dropoff_map = f.Map(location = location, zoom_start = 10,)


hm_wide = HeatMap( dropoff.values,
                     min_opacity= 0.2,
                     radius= 5, blur= 8,
                     max_zoom= 1 
                 )

dropoff_map.add_child(hm_wide)

dropoff_map

The blue coloured saturation is for the different areas of New York for both pickup and dropoff  

In [ ]:
df=df.drop(labels=["tpep_pickup_datetime","tpep_dropoff_datetime"],axis=1)  #Droping the columns

In [ ]:
df.info()  #Overview/Summary of Dataframe

In [ ]:
df.head()

In [ ]:
df['passenger_count'].value_counts()

In [ ]:
df['passenger_count'] = df['passenger_count'].fillna(1)

In [ ]:
df['passenger_count']

In [ ]:
df.isna().sum()

In [ ]:
df["passenger_count"].unique()   #cheking data element in the columns}

In [ ]:
plt.figure(figsize=(16, 6)) 
plt.title('tpep_pickup_datetime_hour')
plt.xlabel('Different Times of the Day')

sb.countplot(x='tpep_pickup_datetime_hour', data=df, palette=("crest"))


In [ ]:
plt.figure(figsize=(16, 6)) 
plt.title('hour')
plt.xlabel('Different Times of the Day')

sb.countplot(x='tpep_dropoff_datetime_hour', data=df, palette=("crest"))


The graphs show count of passengers using the taxi at each hour of the day. It is clear that the city
has high traffic from 8 am  to 11 pm as most people travel to work at these hours and traffic peaks at evening 5 pm to 7 pm.

In [ ]:
plt.figure(figsize=(16, 6)) 
plt.title('Pick-offs on Seven Days of a Week')
plt.xlabel("Days of a Week")

sb.countplot(x='tpep_pickup_datetime_day', data=df, palette=("crest"))


In [ ]:
plt.figure(figsize=(16, 6)) 
plt.title('Drop-offs on Seven Days of a Week')
plt.xlabel('Days of a  Week')

sb.countplot(x='tpep_dropoff_datetime_day', data=df, palette=("crest"))


The graphs show how the traffic varies from weekdays to weekends
0 - Sunday
1 - Monday
2 - Tuesday
3 - Wednesday
4 - Thursady
5 - Friday
6 - Saturday

* Cheking outliers

In [ ]:
sb.boxplot(x=df["passenger_count"])

In [ ]:
sb.boxplot(x=df["trip_duration"])

In [ ]:
df["trip_duration"].value_counts()

In [ ]:
sb.boxplot(x=df["tpep_pickup_datetime_day"])

In [ ]:
sb.boxplot(x=df["tpep_pickup_datetime_month"])

In [ ]:
sb.boxplot(x=df["tpep_pickup_datetime_hour"])

In [ ]:
sb.boxplot(x=df["tpep_pickup_datetime_minute"])

In [ ]:
sb.boxplot(x=df["tpep_dropoff_datetime_day"])

In [ ]:
sb.boxplot(x=df["tpep_dropoff_datetime_month"])

In [ ]:
sb.boxplot(x=df["tpep_dropoff_datetime_hour"])

In [ ]:
sb.boxplot(x=df["tpep_dropoff_datetime_minute"])

In [ ]:
sb.boxplot(x=df["pickup_longitude"])

In [ ]:
sb.boxplot(x=df["pickup_latitude"])

In [ ]:
sb.boxplot(x=df["dropoff_latitude"])

In [ ]:
sb.boxplot(x=df["dropoff_longitude"])

In [ ]:
df.columns

In [ ]:
df=df.drop(labels=['trip_distance','PULocationID', 'DOLocationID',
       'payment_type', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount','total_amount',
       'improvement_surcharge','congestion_surcharge',
       'airport_fee'],axis=1)

df

In [ ]:
df.skew()

skewness before outlier treatment

In [ ]:
def outlier_detect(df):
    for i in df.describe().columns:
        Q1=df.describe().at['25%',i]
        Q3=df.describe().at['75%',i]
        IQR=Q3 - Q1
        LTV=Q1 - 1.5 * IQR
        UTV=Q3 + 1.5 * IQR
        x=np.array(df[i])
        p=[]
        for j in x:
            if j < LTV or j>UTV:
                p.append(df[i].median())
            else:
                p.append(j)
        df[i]=p
    return df

Function for outlier removal : first we teke 25% data as Q1 and 75% data as Q3. Then we define Inter Quartile Range(IQR) as Q3-Q1. 
Then we define Lower Tube Values(LTV) and Upper Tube Values(UTV). 

In [ ]:
new_df=outlier_detect(df)

New_df After removing outliers.

In [ ]:
new_df[["trip_duration"]]

In [ ]:
new_df["passenger_count"].value_counts()

In [ ]:
sb.boxplot(x=new_df["passenger_count"])

In [ ]:
sb.boxplot(x=new_df["trip_duration"])

In [ ]:
sb.boxplot(x=new_df["tpep_pickup_datetime_month"])

In [ ]:
sb.boxplot(x=new_df["tpep_pickup_datetime_day"])

In [ ]:
sb.boxplot(x=new_df["tpep_pickup_datetime_hour"])

In [ ]:
sb.boxplot(x=new_df["tpep_pickup_datetime_minute"])

In [ ]:
sb.boxplot(x=new_df["tpep_dropoff_datetime_day"])

In [ ]:
sb.boxplot(x=new_df["tpep_dropoff_datetime_month"])

In [ ]:
sb.boxplot(x=new_df["tpep_dropoff_datetime_hour"])

In [ ]:
sb.boxplot(x=new_df["tpep_dropoff_datetime_minute"])

In [ ]:
sb.boxplot(x=df["pickup_longitude"])

In [ ]:
sb.boxplot(x=df["pickup_latitude"])

In [ ]:
sb.boxplot(x=df["dropoff_longitude"])

In [ ]:
sb.boxplot(x=df["dropoff_latitude"])

In [ ]:
# cheking skewness

In [ ]:
# new_df.skew()
new_df.skew()

In [ ]:
sb.distplot(new_df['passenger_count'], kde = True, bins = 5) 

In [ ]:
# new_df.shape
new_df.shape

Dividing columns into categorical and continious

In [ ]:
cat=[]
con=[]
for i in new_df:
    if(df[i].dtypes=="object"):
        cat.append(i)
    else:
        con.append(i)

In [ ]:
cat

In [ ]:
new_df.head()

In [ ]:
con

### Model Building

In [ ]:
Y=new_df[["trip_duration"]]                    #Dependent Variable
X=new_df.drop(labels=["trip_duration","zone"],axis=1)  #Independent Variables

X.columns

### Standerdizing the Data 


Standardization is about making sure that data is internally consistant and each data type has same content and format so that all variables are contributing in prediction.

In [ ]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
Xnew=pd.DataFrame(sc.fit_transform(X))

In [ ]:
Xnew 

In [ ]:
Xnew.columns=['passenger_count', 'pickup_longitude', 'pickup_latitude',
       'dropoff_longitude', 'dropoff_latitude', 'tpep_pickup_datetime_day',
       'tpep_pickup_datetime_month', 'tpep_pickup_datetime_hour',
       'tpep_pickup_datetime_minute', 'tpep_dropoff_datetime_day',
       'tpep_dropoff_datetime_month', 'tpep_dropoff_datetime_hour',
       'tpep_dropoff_datetime_minute']
#Adding the colums name

In [ ]:
Xnew

In [ ]:
Xnew.columns   #columns overview

Spliting Data into training and testing set

Split the data into training and testing as 67% and 33% respectively.

In [ ]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest=train_test_split(Xnew,Y,test_size=0.33,random_state=21)

In [ ]:
xtest

In [ ]:
xtrain

# Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression
lr=LinearRegression()
Model=lr.fit(xtrain,ytrain)
pred_tr=Model.predict(xtrain)
pred_ts=Model.predict(xtest)

from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score,accuracy_score
print('MAE:', mean_absolute_error(ytest, pred_ts))
print('MSE:', mean_squared_error(ytest, pred_ts))
print('RMSE:', np.sqrt(mean_squared_error(ytest, pred_ts)))
print('R2', r2_score(ytest,pred_ts))

In [ ]:
# Visualize the performance of the model

import matplotlib.pyplot as plt

plt.plot(xtrain,pred_tr,color='r')
plt.plot(X,Y,'b.')
plt.xlabel("Xtrain")
plt.ylabel("pred_tr")
plt.show()

# Regularization of the model

# Ridge

In [ ]:
tg = {"alpha":[0.991,0.992,0.993,0.994,0.995,0.996,0.997,0.998,0.999,1.001,1.002,1.003,1.004,1.005,1.006,1.007,1.008,1.009]}

In [ ]:
from sklearn.linear_model import Ridge,Lasso
rr = Ridge()



from sklearn.model_selection import GridSearchCV
cv1 = GridSearchCV(rr,tg,scoring="neg_mean_absolute_error",cv=4)
cvmodel1 = cv1.fit(xtrain,ytrain)
cvmodel1.best_params_

In [ ]:
rr = Ridge(alpha=1.009)

model = rr.fit(xtrain,ytrain)

pred_tr = model.predict(xtrain)
pred_ts = model.predict(xtest)

from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score,accuracy_score
print('MAE:', mean_absolute_error(ytest, pred_ts))
print('MSE:', mean_squared_error(ytest, pred_ts))
print('RMSE:', np.sqrt(mean_squared_error(ytest, pred_ts)))
print('R2', r2_score(ytest,pred_ts))


plt.plot(xtrain,pred_tr,color='r')
plt.plot(X,Y,'b.')
plt.xlabel("xtrain")
plt.ylabel("pred_tr")
plt.show()

# Lasso

In [ ]:
ls = Lasso()
from sklearn.model_selection import GridSearchCV
cv2 = GridSearchCV(ls,tg,scoring="neg_mean_absolute_error",cv=4)
cvmodel2 = cv2.fit(xtrain,ytrain)
cvmodel2.best_params_

In [ ]:

ls = Lasso(alpha=1.009)

model = ls.fit(xtrain,ytrain)

pred_tr = model.predict(xtrain)
pred_ts = model.predict(xtest)

from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score,accuracy_score
print('MAE:', mean_absolute_error(ytest, pred_ts))
print('MSE:', mean_squared_error(ytest, pred_ts))
print('RMSE:', np.sqrt(mean_squared_error(ytest, pred_ts)))
print('R2', r2_score(ytest,pred_ts))


plt.plot(xtrain,pred_tr,color='r')
plt.plot(X,Y,'b.')
plt.xlabel("xtrain")
plt.ylabel("pred_tr")
plt.show()

# Polynomial Regression

In [ ]:
from sklearn.preprocessing import PolynomialFeatures  
poly_regs= PolynomialFeatures(degree= 3)  
xtrain_poly= poly_regs.fit_transform(xtrain)
xtest_poly= poly_regs.fit_transform(xtest)
lin_reg_2 =LinearRegression()  
model=lin_reg_2.fit(xtrain_poly,ytrain)  
pred_tr=model.predict(xtrain_poly)
pred_ts=model.predict(xtest_poly)

In [ ]:
from sklearn.metrics import mean_absolute_error,mean_squared_error
print('MAE:', mean_absolute_error(ytest, pred_ts))
print('MSE:', mean_squared_error(ytest, pred_ts))
print('RMSE:', np.sqrt(mean_squared_error(ytest, pred_ts)))
print('R2', r2_score(ytest,pred_ts))


plt.plot(xtrain,pred_tr,color='r')
plt.plot(X,Y,'b.')
plt.xlabel("xtrain")
plt.ylabel("pred_tr")
plt.show()

In [ ]:
lin_reg_2.score(xtest_poly,ytest)

# Support Vector Regression

In [ ]:
# from sklearn.svm import SVR
# svr = SVR(kernel = 'linear')
# model=svr.fit(xtrain,ytrain)
# pred_tr=model.predict(xtrain)
# pred_ts=model.predict(xtest)
# #
# from sklearn.metrics import mean_absolute_error,mean_squared_error
# print('MAE:', mean_absolute_error(ytest, pred_ts))
# print('MSE:', mean_squared_error(ytest, pred_ts))
# print('RMSE:', np.sqrt(mean_squared_error(ytest, pred_ts)))
# print('R2', r2_score(ytest,pred_ts))
# print("------------------------------------------------")
# tr_err=mean_absolute_error(ytrain,pred_tr)
# ts_err=mean_absolute_error(ytest,pred_ts)
# print("Training error=",tr_err)
# print("Testing error=",ts_err)
# print("-------------------------------------------------")

# if(tr_err<ts_err):
#     print("model is overfited")
# else:
#     print("model is underfited")

In [ ]:
# plt.plot(xtrain,pred_tr,color='r')
# plt.plot(X,Y,'b.')
# plt.xlabel("xtrain")
# plt.ylabel("pred_tr")
# plt.show()

# Knn Regression

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
knr = KNeighborsRegressor(n_neighbors=5)
model = knr.fit(xtrain,ytrain)
pred = model.predict(xtest)
from sklearn.metrics import mean_absolute_error,mean_squared_error
print('MAE:', mean_absolute_error(ytest, pred_ts))
print('MSE:', mean_squared_error(ytest, pred_ts))
print('RMSE:', np.sqrt(mean_squared_error(ytest, pred_ts)))
print('R2', r2_score(ytest,pred_ts))


# Hyperparameter tuning for KNeighborsRegressor

In [ ]:
for i in range(2,10,1):
    from sklearn.neighbors import KNeighborsRegressor
    knr = KNeighborsRegressor(n_neighbors=i)
    model = knr.fit(xtrain,ytrain)
    pred = model.predict(xtest)
    
    from sklearn.metrics import mean_absolute_error
    print(i,mean_absolute_error(ytest,pred))

plt.plot(xtrain,pred_tr,color='r')
plt.plot(X,Y,'b.')
plt.xlabel("xtrain")
plt.ylabel("pred_tr")
plt.show()

# Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeRegressor
dtr=DecisionTreeRegressor(max_depth=10,random_state=10)
Model=dtr.fit(xtrain,ytrain)
pred_tr=Model.predict(xtrain)
pred_ts=Model.predict(xtest)

from sklearn.metrics import mean_absolute_error,accuracy_score,r2_score,mean_squared_error
print('MAE:', mean_absolute_error(ytest, pred_ts))
print('MSE:', mean_squared_error(ytest, pred_ts))
print('RMSE:', np.sqrt(mean_squared_error(ytest, pred_ts)))
print('R2', r2_score(ytest,pred_ts))


plt.plot(xtrain,pred_tr,color='r')
plt.plot(X,Y,'b.')
plt.xlabel("xtrain")
plt.ylabel("pred_tr")
plt.show()

In [ ]:
# pip install graphviz

In [ ]:
from graphviz import *
from sklearn.tree import DecisionTreeClassifier, plot_tree
fig, axes = plt.subplots(nrows = 1,ncols = 1,figsize = (4,4), dpi=300)
plot_tree(dtr,
               feature_names = xtrain.columns,
               class_names=new_df.trip_duration,
               filled = True);
fig.savefig('imagename.png')

# Random Forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf=RandomForestRegressor(n_estimators=20,random_state=21)
Model=rf.fit(xtrain,ytrain)
pred=Model.predict(xtrain)
pred_tr=Model.predict(xtrain)
pred_ts=Model.predict(xtest)
from sklearn.metrics import mean_absolute_error,accuracy_score,r2_score,mean_squared_error
print('MAE:', mean_absolute_error(ytest, pred_ts))
print('MSE:', mean_squared_error(ytest, pred_ts))
print('RMSE:', np.sqrt(mean_squared_error(ytest, pred_ts)))
print('R2', r2_score(ytest,pred_ts))


We have MAE as 2.488 and RMSE as 3.799.R2 score is 0.7472 which is close to 1. Thus we can say that the model is predicting with minimal errors and good accuracy.

# Adaboost Regressor

In [ ]:
from sklearn.ensemble import AdaBoostRegressor
adb=AdaBoostRegressor(DecisionTreeRegressor(max_depth=20),n_estimators=500,learning_rate=0.1,random_state=21)
model=adb.fit(xtrain,ytrain)
pred_tr=Model.predict(xtrain)
pred_ts=Model.predict(xtest)
from sklearn.metrics import mean_absolute_error,accuracy_score,r2_score,mean_squared_error
print('MAE:', mean_absolute_error(ytest, pred_ts))
print('MSE:', mean_squared_error(ytest, pred_ts))
print('RMSE:', np.sqrt(mean_squared_error(ytest, pred_ts)))
print('R2', r2_score(ytest,pred_ts))


In [ ]:
pred_tr

# XGboost

In [ ]:
#pip install xgboost

In [ ]:
from xgboost import XGBRegressor #XGboostgressor
xgbr=XGBRegressor()
model=xgbr.fit(xtrain,ytrain)


In [ ]:
pred_tr1=Model.predict(xtrain)
pred_ts1=Model.predict(xtest)
from sklearn.metrics import mean_absolute_error,accuracy_score,r2_score,mean_squared_error
print('MAE:', mean_absolute_error(ytest, pred_ts1))
print('MSE:', mean_squared_error(ytest, pred_ts1))
print('RMSE:', np.sqrt(mean_squared_error(ytest, pred_ts1)))
print('R2', r2_score(ytest,pred_ts1))



# BayesianRidge

In [ ]:
from sklearn.linear_model import BayesianRidge
bysn=BayesianRidge()
model=bysn.fit(xtrain,ytrain)
pred_tr1=Model.predict(xtrain)
pred_ts1=Model.predict(xtest)
from sklearn.metrics import mean_absolute_error,accuracy_score,r2_score,mean_squared_error
print('MAE:', mean_absolute_error(ytest, pred_ts1))
print('MSE:', mean_squared_error(ytest, pred_ts1))
print('RMSE:', np.sqrt(mean_squared_error(ytest, pred_ts1)))
print('R2', r2_score(ytest,pred_ts1))


In [ ]:
ytest

In [ ]:
ytest.value_counts()

In [ ]:
pred_ts1

### create a model pickle file

In [ ]:
# Random Forest_model
# pickle.dump(rf, open('r_model.pkl', 'wb'))
import pickle
pickle.dump(rf, open('R_model.pkl', 'wb'))

 Unique values columns

In [ ]:
fdf=pd.read_csv("final_yellow_tripdata.csv")
fdf

In [ ]:
pickup_longitude=pd.DataFrame(fdf.pickup_longitude.unique())
pickup_longitude.columns=["pickup_longitude"]
pickup_longitude.to_csv('final_taxi_data_pickup_longitude.csv')
pickup_longitude.head()

In [ ]:
pickup_latitude=pd.DataFrame(fdf.pickup_latitude.unique())
pickup_latitude.columns=["pickup_latitude"]
pickup_latitude.to_csv('final_taxi_data_pickup_latitude.csv')
pickup_latitude.head()

In [ ]:
dropoff_longitude=pd.DataFrame(df.dropoff_longitude.unique())
dropoff_longitude.columns=["dropoff_longitude"]
dropoff_longitude.to_csv('final_taxi_data_dropoff_longitude.csv')
dropoff_longitude.head()

In [ ]:
dropoff_latitude=pd.DataFrame(df.dropoff_latitude.unique())
dropoff_latitude.columns=["dropoff_latitude"]
dropoff_latitude.to_csv('final_taxi_data_dropoff_latitude.csv')
dropoff_latitude.head()

In [ ]:
C=pd.DataFrame(data['tpep_dropoff_datetime'].dt.time)
C=C.sort_values(by='tpep_dropoff_datetime',ascending=True)
C['tpep_dropoff_datetime']=A['tpep_dropoff_datetime'].apply(lambda t: t.replace(second=0))

D=( C.sort_values('tpep_dropoff_datetime')
    .drop_duplicates('tpep_dropoff_datetime',keep = 'first')
    .reset_index(drop=True)
    .assign(a = lambda x: x.index + 1) )

D=pd.DataFrame(B['tpep_dropoff_datetime'])

D.to_csv("dk_time.csv")
D